<a href="https://colab.research.google.com/github/omkarakolkar/machine_learning_projects/blob/main/P5_Face_mask_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
from keras.preprocessing import image
import cv2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!cp "/content/drive/MyDrive/Machine Learning project /datasets/dataset_facemask/archive.zip" /content/

In [ ]:
!rm -rf /content/images

In [ ]:
!unzip "/content/archive.zip" -d /content/images

Streaming output truncated to the last 5000 lines.
  inflating: /content/images/data/with_mask/with_mask_3297.jpg  
  inflating: /content/images/data/with_mask/with_mask_3298.jpg  
  inflating: /content/images/data/with_mask/with_mask_3299.jpg  
  inflating: /content/images/data/with_mask/with_mask_33.jpg  
  inflating: /content/images/data/with_mask/with_mask_330.jpg  
  inflating: /content/images/data/with_mask/with_mask_3300.jpg  
  inflating: /content/images/data/with_mask/with_mask_3301.jpg  
  inflating: /content/images/data/with_mask/with_mask_3302.jpg  
  inflating: /content/images/data/with_mask/with_mask_3303.jpg  
  inflating: /content/images/data/with_mask/with_mask_3304.jpg  
  inflating: /content/images/data/with_mask/with_mask_3305.jpg  
  inflating: /content/images/data/with_mask/with_mask_3306.jpg  
  inflating: /content/images/data/with_mask/with_mask_3307.jpg  
  inflating: /content/images/data/with_mask/with_mask_3308.jpg  
  inflating: /content/images/data/with_mas

In [ ]:
!ls -R /content/images/

/content/images/:
data

/content/images/data:
with_mask  without_mask

/content/images/data/with_mask:
with_mask_1000.jpg  with_mask_1840.jpg	with_mask_267.jpg   with_mask_3518.jpg
with_mask_1001.jpg  with_mask_1841.jpg	with_mask_2680.jpg  with_mask_3519.jpg
with_mask_1002.jpg  with_mask_1842.jpg	with_mask_2681.jpg  with_mask_351.jpg
with_mask_1003.jpg  with_mask_1843.jpg	with_mask_2682.jpg  with_mask_3520.jpg
with_mask_1004.jpg  with_mask_1844.jpg	with_mask_2683.jpg  with_mask_3521.jpg
with_mask_1005.jpg  with_mask_1845.jpg	with_mask_2684.jpg  with_mask_3522.jpg
with_mask_1006.jpg  with_mask_1846.jpg	with_mask_2685.jpg  with_mask_3523.jpg
with_mask_1007.jpg  with_mask_1847.jpg	with_mask_2686.jpg  with_mask_3524.jpg
with_mask_1008.jpg  with_mask_1848.jpg	with_mask_2687.jpg  with_mask_3525.jpg
with_mask_1009.jpg  with_mask_1849.jpg	with_mask_2688.jpg  with_mask_3526.jpg
with_mask_100.jpg   with_mask_184.jpg	with_mask_2689.jpg  with_mask_3527.jpg
with_mask_1010.jpg  with_mask_1850.jpg	wi

In [ ]:
!ls /content/images/

data


In [ ]:
import zipfile

zip_path = "/content/archive.zip"  # the copied file
base_path = "/content/images/data/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(base_path)

print("Extraction complete!")


Extraction complete!


In [ ]:
categories = ['with_mask', 'without_mask']

data =[]
for category in categories:
    folder_path = os.path.join(base_path, category)

    label = categories.index(category)

    for file in os.listdir(folder_path):
        img_path = os.path.join(folder_path,file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))

        data.append([img,label]) #labeling the images

In [ ]:
len(data)

7553

In [ ]:
import random
random.shuffle(data)

In [ ]:
x = []
y = []

for features, label in data:
  x.append(features)
  y.append(label)

In [ ]:
len(x)

7553

In [ ]:
len(y)

7553

In [ ]:
import numpy as np

In [ ]:
x = np.array(x)
y = np.array(y)

In [ ]:
x = x/255 #standardising x

In [ ]:
import tensorflow as tf

batch_size = 32
img_size = (224, 224)  # Resize all images

# Training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    base_path,
    validation_split=0.2,       # split 80/20
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

# Validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    base_path,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

Found 15106 files belonging to 3 classes.
Using 12085 files for training.
Found 15106 files belonging to 3 classes.
Using 3021 files for validation.


In [ ]:
#performance optimizations
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Model


In [ ]:
from keras.applications.vgg16 import VGG16
vgg = VGG16()

553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step


In [ ]:
from keras import Sequential
model = Sequential()

In [ ]:
for layer in vgg.layers[:-1]:
  model.add(layer)

In [ ]:
for layer in model.layers:
  layer.trainable=False

In [ ]:
from keras.layers import Dense

In [ ]:
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         4,097 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,264,641 (512.18 MB)

 Trainable params: 4,097 (16.00 KB)

 Non-trainable params: 134,260,544 (512.16 MB)

In [ ]:
#### Dont run this.
# from keras.applications.vgg16 import VGG16
# from keras.models import Sequential
# from keras.layers import Dense, GlobalAveragePooling2D

# vgg = VGG16(include_top=False, input_shape=(224,224,3))

# model = Sequential()
# model.add(vgg)
# model.add(GlobalAveragePooling2D())  # replaces the flatten + dense(4096) layers
# model.add(Dense(1, activation='sigmoid'))  # binary classification

# # freeze pretrained layers
# for layer in vgg.layers:
#     layer.trainable = False

# model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
)

Epoch 1/5
1511/1511 ━━━━━━━━━━━━━━━━━━━━ 118s 73ms/step - accuracy: 0.2831 - loss: 1.1292 - val_accuracy: 0.2347 - val_loss: 0.9426
Epoch 2/5
1511/1511 ━━━━━━━━━━━━━━━━━━━━ 132s 70ms/step - accuracy: 0.2510 - loss: 0.7017 - val_accuracy: 0.2281 - val_loss: 0.7648
Epoch 3/5
1511/1511 ━━━━━━━━━━━━━━━━━━━━ 162s 84ms/step - accuracy: 0.2486 - loss: 0.5979 - val_accuracy: 0.2195 - val_loss: 0.6861
Epoch 4/5
1511/1511 ━━━━━━━━━━━━━━━━━━━━ 122s 70ms/step - accuracy: 0.2523 - loss: 0.5611 - val_accuracy: 0.2393 - val_loss: 0.9745
Epoch 5/5
1511/1511 ━━━━━━━━━━━━━━━━━━━━ 142s 71ms/step - accuracy: 0.2550 - loss: 0.5678 - val_accuracy: 0.2383 - val_loss: 0.8986


In [3]:
cap = cv2.VideoCapture(0)

In [6]:
while True:

  ret, frame = cap.read()

  cv2.imshow("window", frame)

  if cv2.waitKey(1) & 0xFF == ord('x'):
    break

cv2.destroy()

In [ ]:
def detect_face_mask(img):

  y_pred = model.predict(img.reshape(1,224,224,3))

  return y_pred